# **SO SÁNH MỨC GIỐNG NHAU GIỮA HAI CÂU VỀ MẶT NGỮ NGHĨA**

In [2]:
:dep candle-core = { version = "0.9.1", features = ["cuda", "cudnn"] }
:dep candle-nn = { version = "0.9.1", features = ["cuda", "cudnn"] }
:dep candle-transformers = { version = "0.9.1", features = ["cuda", "cudnn"] }
:dep hf-hub = { version = "0.4.3", features = ["native-tls", "tokio"] }
:dep serde_json = "1.0.145"
:dep tokenizers = "0.22.1"

In [3]:
use candle_core::{DType, Device, Tensor, IndexOp};
use candle_nn::VarBuilder;
use candle_transformers::models::bert::{BertModel, Config as BertConfig};
use hf_hub::{api::sync::ApiBuilder, Repo, RepoType};
use tokenizers::{
    DecoderWrapper, ModelWrapper, Tokenizer
    PostProcessorWrapper, PreTokenizerWrapper,
    TokenizerImpl, NormalizerWrapper
};

In [4]:
pub fn build_model_and_tokenizer(
    model_id: String, revision: String, device: &Device
) -> Result<(BertModel, Tokenizer), Box<dyn std::error::Error>>
{
    // [01]
    let access_token = std::env::var("HF_TOKEN").map_err(|e| format!("{e}"))?;
    let api = ApiBuilder::new()
        .with_progress(true)
        .with_token(Some(access_token))
        .build()?;

    // [02]
    let repo = Repo::with_revision(model_id, RepoType::Model, revision);
    let api = api.repo(repo);
    let config_file = api.get("config.json")?;
    let tokenizer_file = api.get("tokenizer.json")?;

    // [03]
    let config: String = std::fs::read_to_string(config_file)?;
    let config: BertConfig = serde_json::from_str(&config)?;

    // [04]
    let tokenizer: Tokenizer = Tokenizer::from_file(tokenizer_file)
                                                .map_err(|e| format!("{e}"))?;
    // [05]
    let vb = match api.get("model.safetensors") {
        Ok(path) => {
            println!("Sử dụng SafeTensors: {:?}", path);
            unsafe {
                VarBuilder::from_mmaped_safetensors(
                    &[path], DType::F32, device
                )
            }
        },
        Err(_) => {
            let path = api.get("pytorch_model.bin")?;
            println!("Sử dụng PyTorch Binary: {:?}", path);
            VarBuilder::from_pth(&path, DType::F32, device)
        }
    }?;

    // [06]
    let model = BertModel::load(vb, &config)?;
    
    Ok((model, tokenizer))
}

In [5]:
pub fn gen_embeddings_mean_pooling(
    tokenizer: &mut TokenizerImpl<
        ModelWrapper, NormalizerWrapper,
        PreTokenizerWrapper, PostProcessorWrapper, DecoderWrapper,
    >,
    model: &BertModel,
    prompt: &str,
) -> Result<Tensor, Box<dyn std::error::Error>> {
    // [01]
    let device = &model.device;

    // [02]
    let tokens: Vec<u32> = tokenizer.encode(prompt, true)
        .map_err(|e| format!("{e}"))?
        .get_ids()
        .to_vec();

    // [03]
    let token_ids = Tensor::new(&tokens[..], device)?.unsqueeze(0)?;
    //println!("-- {:?}\n{}", token_ids.shape(), token_ids.narrow(1, 0, 5)?);

    // [04]
    let token_type_ids = token_ids.zeros_like()?;
    /*println!("-- {:?}\n{}",
        token_type_ids.shape(), token_type_ids.narrow(1, 0, 5)?
    );*/

    let emb: Tensor = model.forward(&token_ids, &token_type_ids, None)?;
    //println!("Trước pooling -- {:?}\n{}", emb.shape(), emb.narrow(1, 0, 5)?);

    // [05]
    let (_n_sentence, n_tokens, _hidden_size) = emb.dims3()?;
    let emb: Tensor = (emb.sum(1)? / (n_tokens as f64))?;
    //println!("Sau pooling -- {:?}\n{}", emb.shape(), emb.narrow(1, 0, 5)?);

    // [06]
    let normalized_emb = normalize_l2(&emb)?;
    /*println!("-- {:?}\n{}",
        normalized_emb.shape(), normalized_emb.narrow(1, 0, 5)?
    );*/

    Ok(normalized_emb)
}

/// Hàm thực hiện chuẩn hóa một ten-xơ bằng phương thức L2-Norm
pub fn normalize_l2(v: &Tensor) -> Result<Tensor, Box<dyn std::error::Error>> {
    Ok(v.broadcast_div(&v.sqr()?.sum_keepdim(1)?.sqrt()?)?)
}

In [6]:
const MODEL: &str =
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2";
const REVISION: &str = "main";

const SENTENCES: [[&str; 2]; 9] = [
    ["this is a Rust programming teaching book",
    "đây là một cuốn sách dạy lập trình Rust"],
    ["tôi rất thích đi biển vào mùa hè",
    "I like going to the beach in the summer very much"],
    ["học lập trình sẽ phát triển tư duy lô gíc",
    "tư duy lô gíc sẽ được cải thiện khi học lập trình"],
    ["hôm qua, thủ tướng đã sang thăm Cuba",
    "The Priminister visited Cuba yesterday"],
    ["hôm qua, thủ tướng đã sang thăm Cuba",
    "hôm qua, người đứng đầu nhà nước đã sang thăm Cuba"],
    ["cảnh sát đang truy đuổi bọn tội phạm",
    "bọn tội phạm đang truy đuổi cảnh sát"],
    ["bọn tội phạm đang bị cảnh sát truy đuổi",
    "cảnh sát đang đuổi bắt bọn tội phạm"],
    ["the police are chasing the criminals",
    "cảnh sát không hề truy đuổi bọn tội phạm"],
    ["khi xảy ra rủi ro, bạn sẽ được chi trả quyền lợi bảo hiểm",
    "bảo hiểm đặc biệt có ý nghĩa khi bạn gặp rủi ro"]
];

{
    // [01]
    let device = match Device::cuda_if_available(0) {
        Ok(cuda) => cuda,
        _ => Device::Cpu
    };

    // [02]
    let (model, mut tokenizer) = build_model_and_tokenizer(
                                    MODEL.to_string(), REVISION.to_string(),
                                    &device)?;
    // [03]
    let tokenizer = tokenizer
        .with_padding(None)
        .with_truncation(None)
        .map_err(|e| format!("Lỗi cấu hình tokenizer: {e}"))?;
    /*
    let test_tokenizer = tokenizer.encode(SENTENCES[0][1], false)?;
    println!("{:?}", test_tokenizer.get_tokens());
    println!("{:?}", test_tokenizer.get_ids());*/
    
    // [04]
    let mut sentences_similarity_mean = Vec::<f32>::new();
    for s in SENTENCES {
        let emb1 = gen_embeddings_mean_pooling(tokenizer, &model, s[0])?;
        let emb2 = gen_embeddings_mean_pooling(tokenizer, &model, s[1])?;
        //println!("emb1: {:?}", emb1);
        //println!("emb2: {:?}", emb2);

        let sum = &emb1
            .matmul(&emb2.transpose(0, 1)?)?.sum_all()?
            .to_scalar::<f32>()?;

        sentences_similarity_mean.push(*sum);
    }

    // [05]
    for (i, s) in SENTENCES.iter().enumerate() {
        println!("-----");
        println!("| {}", s[0]);
        println!("| {}", s[1]);
        println!("-----> Mức độ giống nhau: {}",
            sentences_similarity_mean.get(i).unwrap()
        );
    }
}

Sử dụng SafeTensors: "/home/trinhtuan/.cache/huggingface/hub/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/4328cf26390c98c5e3c738b4460a05b95f4911f5/model.safetensors"
-----
| this is a Rust programming teaching book
| đây là một cuốn sách dạy lập trình Rust
-----> Mức độ giống nhau: 0.95408356
-----
| tôi rất thích đi biển vào mùa hè
| I like going to the beach in the summer very much
-----> Mức độ giống nhau: 0.9099325
-----
| học lập trình sẽ phát triển tư duy lô gíc
| tư duy lô gíc sẽ được cải thiện khi học lập trình
-----> Mức độ giống nhau: 0.6341647
-----
| hôm qua, thủ tướng đã sang thăm Cuba
| The Priminister visited Cuba yesterday
-----> Mức độ giống nhau: 0.89707315
-----
| hôm qua, thủ tướng đã sang thăm Cuba
| hôm qua, người đứng đầu nhà nước đã sang thăm Cuba
-----> Mức độ giống nhau: 0.9215536
-----
| cảnh sát đang truy đuổi bọn tội phạm
| bọn tội phạm đang truy đuổi cảnh sát
-----> Mức độ giống nhau: 0.8732573
-----
| bọn tội phạm đang bị

()